## NOMS Paper

In [ ]:
import Pkg; Pkg.activate(".")

In [ ]:
using Revise

In [ ]:
using ArgCheck
using Distributions
using HMMBase
using ParsimoniousMonitoring
using ProgressMeter
using PyPlot
using Random

In [ ]:
using POMDPs
using POMDPModelTools
using POMDPSimulators
using DiscreteValueIteration

In [ ]:
# TODO: Implement only one route in receding horizon

### 5.1 Synthetic scenarios

In [ ]:
MRE(reference, candidate) = mean((candidate .- reference) ./ reference);

In [ ]:
smdp = SparseTabularMDP(mdp);

In [ ]:
length(states(smdp))

**Two paths**

**Three paths**

In [ ]:
p1 = HMM([0.9 0.1; 0.1 0.9], [Constant(1.0), Constant(3.0)])
p2 = HMM([0.8 0.2; 0.3 0.7], [Constant(0.5), Constant(4.0)])
p3 = HMM([0.65 0.35; 0.35 0.65], [Constant(0.25), Constant(3.75)])
mdp = MonitoringMDP([p1, p2, p3], [20, 10, 10], [0.5, 0.5, 0.5]);

In [ ]:
@time smdp = SparseTabularMDP(mdp);

In [ ]:
length(states(mdp))

In [ ]:
policy = RecedingHorizonPolicy(mdp, 3);

In [ ]:
@showprogress for state in states(mdp)
    action(policy, state)
end

### 8.1 A first simple example

In [ ]:
# A discrete probability distribution with a single value.
constdist(x) = DiscreteNonParametric([x], [1.0])

In [ ]:
# Deterministic path
p1 = HMM(ones(1,1), [constdist(8.0)])
# Stochatich path
p2 = HMM([0.99 0.01; 0.02 0.98], [constdist(5.0), constdist(10.0)])
# τmax = 150, c = 0.65
mdp = MonitoringMDP([p1, p2], [150, 150], [0, 0.65])
smdp = SparseTabularMDP(mdp);

In [ ]:
data = hcat(rand(mdp.models[1], 3000), rand(mdp.models[2], 3000));

In [ ]:
fig, ax = subplots(figsize = (10, 3))
ax.plot(data[:,1], label = "Deterministic path")
ax.plot(data[:,2], label = "Stochastic path")
ax.set(xlabel = "Timestep", ylabel = L"$L(t)$", ylim = (4, 12))
ax.legend();

#### Greedy policy

Since there is only one stochastic path with two states, we can compute the greedy threshold policy analytically:

In [ ]:
function thresholds(mdp::MonitoringMDP{2})
    @argcheck size(mdp.models[1], 1) == 1 # Deterministic link
    @argcheck size(mdp.models[2], 1) == 2 # Stochastic link
    c = mdp.costs[2]
    l = mean(mdp.models[1].B[1])
    l0, l1 = mean.(mdp.models[2].B)
    c / (l - l0), 1 - c / (l1 - l)
end;

In [ ]:
xmin, xmax = thresholds(mdp)

Here we benchmark against a generic MDP greedy policy, and we verify that it matches the analytical thresholds.

In [ ]:
logbook_greedy = benchmark(mdp, GreedyPolicy(mdp), data);

In [ ]:
instants = findall(map(h -> h.a[2], logbook_greedy));

In [ ]:
fig, ax = subplots(figsize = (10, 3))
ax.plot(data[:,1], label = "Deterministic path")
ax.plot(data[:,2], label = "Stochastic path")
ax.scatter(instants, data[instants,2], c = "red", marker = "o")
ax.set(xlabel = "Timestep", ylabel = L"$L(t)$", ylim = (4, 12))
ax.legend();

In [ ]:
predictor = map(logbook_greedy) do history
    state = history.s[2]
    (mdp.models[2].A^(state.timesteps+1))[state.laststate,1]
end;

In [ ]:
fig, ax = subplots(figsize = (10, 3))
ax.plot(predictor)
ax.axhline(xmin, c = "black", ls = "--", lw = 1.0, label = "xmin")
ax.axhline(xmax, c = "black", ls = "--", lw = 1.0, label = "xmax")
ax.set(xlabel = "Timestep", ylabel = L"γ_{t-1,t}(1)", ylim = (0, 1.0))
ax.legend(loc = "upper right");

In [ ]:
# TODO
# # In this case the belief space is a line [0,1] which represents 
# # the probability of the stochastic path being in state 1.
# policy = GreedyPolicy(mdp)
# greedy_actions = map(states(mdp)) do state
#     action(policy, state), (mdp.models[2].A^state[2].timesteps)[state[2].laststate,1]
# end

# # policy = map(states(mdp))

# # # Order the policy by belief values, and find the thresholds
# # perm = sortperm(belief_1d)
# # sorted_belief, sorted_policy = belief_1d[perm], policy.policy[perm]
# # sorted_belief[findall(sorted_policy[2:end] .!= sorted_policy[1:end-1]) .+ 1]

#### MDP policy

In [ ]:
solver = SparseValueIterationSolver(max_iterations=5000, belres=1e-6);

In [ ]:
policy_mdp_99 = solve_sparse(solver, mdp, smdp, 0.99);
logbook_mdp_99 = benchmark(mdp, policy_mdp_99, data);

#### Baseline policies

In [ ]:
logbook_never = benchmark(mdp, never_measure_policy(2), data)
logbook_always = benchmark(mdp, always_measure_policy(2), data);

#### Comparison

$\tilde{G} = 1_{C(t)=1}(l - L(t)) - c 1_{M(t)=1}$

In [ ]:
function gain(mdp::MonitoringMDP, logbook)
    @argcheck size(mdp.models[1], 1) == 1 # Deterministic link
    @argcheck size(mdp.models[2], 1) == 2 # Stochastic link
    c = mdp.costs[2]
    l = mean(mdp.models[1].B[1])
    map(logbook) do history
        ((history.path == 2) * (l - history.delay))
    end
end

function penalized_gain(mdp::MonitoringMDP, logbook)
    @argcheck size(mdp.models[1], 1) == 1 # Deterministic link
    @argcheck size(mdp.models[2], 1) == 2 # Stochastic link
    c = mdp.costs[2]
    l = mean(mdp.models[1].B[1])
    map(logbook) do history
        ((history.path == 2) * (l - history.delay)) - (c * history.a[2])
    end
end

Why do we gain something when we never measure?  
=> Because on average the stochastic path is shorter: 7.5ms vs 8ms.

In [ ]:
fig, ax = subplots(figsize = (10, 3))
ax.plot(cumsum(penalized_gain(mdp, logbook_never)), label = "Never measure")
ax.plot(cumsum(penalized_gain(mdp, logbook_always)), label = "Always measure")
ax.plot(cumsum(penalized_gain(mdp, logbook_greedy)), label = "Greedy policy")
ax.plot(cumsum(penalized_gain(mdp, logbook_mdp_99)), label = "MDP 0.99")
ax.set(xlabel = "Timestep", ylabel = "Cumulative penalized gain")
ax.legend(loc = "upper right")
ax.grid();

In [ ]:
fig, ax = subplots(figsize = (10, 3))
ax.plot(cumsum(gain(mdp, logbook_never)), label = "Never measure")
ax.plot(cumsum(gain(mdp, logbook_always)), label = "Always measure")
ax.plot(cumsum(gain(mdp, logbook_greedy)), label = "Greedy policy")
ax.plot(cumsum(gain(mdp, logbook_mdp_99)), label = "MDP 0.99")
ax.set(xlabel = "Timestep", ylabel = "Cumulative gain")
ax.legend(loc = "upper right")
ax.grid();

#### Monte Carlo simulations

In [ ]:
policy_mdp_01 = solve_sparse(solver, mdp, smdp, 0.01);
policy_mdp_50 = solve_sparse(solver, mdp, smdp, 0.50);
policy_mdp_99 = solve_sparse(solver, mdp, smdp, 0.99);

In [ ]:
policies = Dict(
    "Never measure"  => never_measure_policy(2),
    "Always measure" => always_measure_policy(2),
    "Greedy policy"  => GreedyPolicy(mdp),
    "MDP 0.01" => policy_mdp_01,
    "MDP 0.50" => policy_mdp_50,
    "MDP 0.99" => policy_mdp_99,
);

In [ ]:
function simple_average(logbooks)
    Dict(
        "Average Measures"  => mean(logbook -> sum(h -> h.a[2], logbook), logbooks),
        "Average Penalized Gain" => mean(logbook -> mean(penalized_gain(mdp, logbook)), logbooks)
    )
end

In [ ]:
benchmark_mc(mdp, policies, 100, 3000, summary_fn = simple_average)

In [ ]:
fig, ax = subplots(figsize = (3, 1.0))
ax.scatter(belief_1d, ones(length(belief_1d)), c = policy.policy, s = 1.0)
ax.axvline.([xmin, xmax], lw = 1.0);

### NOMS paper

Dire dans la these qu'on peut appliquer RH en ligne puisque pas besoin de visiter tout les états.

In [ ]:
using ParsimoniousMonitoring: OnlineRecedingHorizonPolicy

In [ ]:
hr = HistoryRecorder(max_steps = 3000, show_progress = true)
s0 = rand(states(mdp))
h_always = simulate(hr, mdp, ConstantPolicy((false,true)), s0);
h_never = simulate(hr, mdp, ConstantPolicy((false,false)), s0);
h_mdp = simulate(hr, mdp, policy, s0);
h_rh = simulate(hr, mdp,  OnlineRecedingHorizonPolicy(mdp, 4), s0);

In [ ]:
figure(figsize=(4,4))
plot(cumsum(map(x -> x[:r], h_always.hist)))
plot(cumsum(map(x -> x[:r], h_never.hist)))
plot(cumsum(map(x -> x[:r], h_mdp.hist)))
plot(cumsum(map(x -> x[:r], h_rh.hist)))

In [ ]:
# function belief_1d(mdp::MonitoringMDP, p::Int, k::Int)
#     states_ = states(mdp)
#     belief = Vector{Float64}(undef, length(states_))
#     model = mdp.models[p]
#     for (i, state) in enumerate(states_)
#         belief[i] = (model.A^state[p].timesteps)[state[p].laststate,k]
#     end
#     belief
# end

In [ ]:
belief = belief_1d(mdp, 2, 1)
fig, ax = subplots(figsize = (3, 1.0))
ax.scatter(belief, ones(length(belief)), c = policy.policy, s = 1.0)

In [ ]:
hr = HistoryRecorder(max_steps=3000)
s0 = rand(states(mdp))
h_always = simulate(hr, mdp, ConstantPolicy((false,true)), s0);
h_never = simulate(hr, mdp, ConstantPolicy((false,false)), s0);
h_mdp = simulate(hr, mdp, policy, s0);

In [ ]:
plot(cumsum(map(x -> x[:r], h_always.hist)))
plot(cumsum(map(x -> x[:r], h_never.hist)))
plot(cumsum(map(x -> x[:r], h_mdp.hist)))

In [ ]:
rand(states(mdp))

In [ ]:
function belief_1d(mdp::MonitoringMDP, p::Int, k::Int)
    states_ = states(mdp)
    belief = Vector{Float64}(undef, length(states_))
    model = mdp.models[p]
    for (i, (state)) in enumerate(states_)
        timesteps, laststate = getstate(state)[p]
        belief[i] = (model.A^timesteps)[laststate,k]
    end
    belief
end

In [ ]:
# TODO: Plot value function

In [ ]:
belief = belief_1d(mdp, 2, 1)
fig, ax = subplots(figsize = (3, 1.0))
ax.scatter(belief, ones(length(belief)), c = res.policy, s = 1.0)

In [ ]:
struct ConstantPolicy <: Policy
    action::CartesianIndex
end
POMDPs.action(policy::ConstantPolicy, _) = policy.action

In [ ]:
struct MDPPolicy <: Policy
    mdp::MonitoringMDP
    policy::Vector{Int}
end

function MDPPolicy(mdp::MonitoringMDP, policy::ValueIterationPolicy)
    MDPPolicy(mdp, policy.policy)
end

function POMDPs.action(policy::MDPPolicy, s)
    state = stateindex(mdp, s)
    action = policy.policy[state]
    actions(mdp)[action]
end

In [ ]:
# pol = ConstantPolicy(CartesianIndex(1,1))
# pol = MDPPolicy(mdp, res);

In [ ]:
# rs = RolloutSimulator(max_steps=10)
# r = simulate(rs, mdp, pol, rand(mdp.states))

In [ ]:
s0 = rand(mdp.states);
# s0 = CartesianIndex(0, 1, 0, 1);

In [ ]:
hr = HistoryRecorder(max_steps=3000)
h_always = simulate(hr, mdp, ConstantPolicy(CartesianIndex(0,1)), s0);
h_never = simulate(hr, mdp, ConstantPolicy(CartesianIndex(0,0)), s0);
h_mdp = simulate(hr, mdp, MDPPolicy(mdp, res), s0);

In [ ]:
sum(map(x -> x[:a] == CartesianIndex(0,1), h_mdp.hist))

In [ ]:
mean(map(x -> x[:r], h_mdp.hist))

In [ ]:
plot(cumsum(map(x -> x[:r], h_always.hist)))
plot(cumsum(map(x -> x[:r], h_never.hist)))
plot(cumsum(map(x -> x[:r], h_mdp.hist)))

https://github.com/JuliaPOMDP/POMDPExamples.jl/blob/master/notebooks/Defining-a-Heuristic-Policy.ipynb

In [ ]:
h_greedy[1]

In [ ]:
x, y = [], []
for (i, action) in enumerate(res.policy)
    action = getaction(actions(mdp)[action])
    timesteps, laststate = getstate(states(mdp)[i])[2]
    push!(x, (p2.A^timesteps)[laststate,1])
    push!(y, action[2])
end
scatter(x, y)

In [ ]:
x, y = [], []
for (i, action) in enumerate(res.policy)
    action = getaction(actions(mdp)[action])
    timesteps, laststate = getstate(states(mdp)[i])[2]
    push!(x, (p2.A^timesteps)[laststate,1])
    push!(y, action[2])
end
scatter(x, y)

#### 8.2 Two Markov chains of two states each

In [ ]:
# TODO: Use DiscreteNonParametric instead of 0-variance Normal distn.
p1 = HMM([0.7 0.3; 0.3 0.7], [Normal(0.5, 0), Normal(2.0, 0)])
p2 = HMM([0.9 0.1; 0.1 0.9], [Normal(1.0,0), Normal(3.0,0)])
mdp = MonitoringMDP([100, 100], [p1, p2], [0.05, 0.15], 0.01);

In [ ]:
smdp = SparseTabularMDP(mdp);

In [ ]:
solver = SparseValueIterationSolver(max_iterations=100, belres=1e-6, verbose=true)
res = solve(solver, smdp);

In [ ]:
x, y, z = [], [], []
for (i, action) in enumerate(res.policy)
    state = getstate(states(mdp)[i])
    timesteps, laststate = state[1]
    push!(x, (p1.A^(timesteps+1))[laststate,1])
    timesteps, laststate = state[2]
    push!(y, (p2.A^(timesteps+1))[laststate,1])
    push!(z, action)
end
# scatter(x, y)

In [ ]:
scatter(x, y, c=z)
xlim(0,1)
ylim(0,1)

TODO: Implement https://juliapomdp.github.io/POMDPModelTools.jl/latest/visualization.html

## Simulation

https://juliapomdp.github.io/POMDPSimulators.jl/stable/parallel/#Parallel-1